# Experimenting with Simulation Parameters

This notebook explores how different parameters affect simulation outcomes.

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cascabel.models.waitline import WaitLine
from cascabel.models.simulation import Simulation
from cascabel.models.models import BorderCrossingConfig, SimulationConfig

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

## Helper Function for Running Experiments

In [ ]:
def run_experiment(arrival_rate, service_rate, num_queues=1, nodes_per_queue=[1]):
    """Run a simulation with given parameters and return key metrics."""
    
    waitline = WaitLine(
        geojson_path="cascabel/paths/usa2mx/bota.geojson",
        speed_regime={"slow": 0.8, "fast": 0.2},
        line_length_seed=0.5
    )
    
    border_config = BorderCrossingConfig(
        num_queues=num_queues,
        nodes_per_queue=nodes_per_queue,
        arrival_rate=arrival_rate,
        service_rates=[service_rate] * sum(nodes_per_queue),
        queue_assignment='shortest',
        safe_distance=8.0,
        max_queue_length=100
    )
    
    simulation_config = SimulationConfig(
        max_simulation_time=3600.0,
        time_factor=1.0,
        enable_telemetry=False,
        enable_position_tracking=False
    )
    
    simulation = Simulation(
        waitline=waitline,
        border_config=border_config,
        simulation_config=simulation_config
    )
    
    simulation()
    stats = simulation.get_statistics()
    
    return {
        'arrival_rate': arrival_rate,
        'service_rate': service_rate,
        'total_arrivals': stats.execution_stats.total_arrivals,
        'total_completions': stats.execution_stats.total_completions,
        'avg_queue_length': stats.execution_stats.average_queue_length,
        'avg_wait_time': stats.execution_stats.average_wait_time,
        'utilization': stats.execution_stats.overall_utilization,
        'throughput': stats.execution_stats.total_completions / stats.simulation_duration * 3600  # per hour
    }

## Experiment 1: Varying Arrival Rates

How does increasing arrival rate affect wait times and queue lengths?

In [ ]:
# Test different arrival rates
arrival_rates = [0.2, 0.5, 1.0, 1.5, 2.0, 2.5]
service_rate = 0.8  # fixed

results = []
for rate in arrival_rates:
    print(f"Running simulation with arrival rate {rate}...")
    result = run_experiment(rate, service_rate)
    results.append(result)

df_arrival = pd.DataFrame(results)
df_arrival

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0,0].plot(df_arrival['arrival_rate'], df_arrival['avg_wait_time'], 'bo-')
axes[0,0].set_xlabel('Arrival Rate (cars/min)')
axes[0,0].set_ylabel('Average Wait Time (s)')
axes[0,0].set_title('Wait Time vs Arrival Rate')
axes[0,0].grid(True)

axes[0,1].plot(df_arrival['arrival_rate'], df_arrival['avg_queue_length'], 'ro-')
axes[0,1].set_xlabel('Arrival Rate (cars/min)')
axes[0,1].set_ylabel('Average Queue Length')
axes[0,1].set_title('Queue Length vs Arrival Rate')
axes[0,1].grid(True)

axes[1,0].plot(df_arrival['arrival_rate'], df_arrival['utilization'], 'go-')
axes[1,0].set_xlabel('Arrival Rate (cars/min)')
axes[1,0].set_ylabel('Utilization')
axes[1,0].set_title('Utilization vs Arrival Rate')
axes[1,0].grid(True)

axes[1,1].plot(df_arrival['arrival_rate'], df_arrival['throughput'], 'mo-')
axes[1,1].set_xlabel('Arrival Rate (cars/min)')
axes[1,1].set_ylabel('Throughput (cars/hour)')
axes[1,1].set_title('Throughput vs Arrival Rate')
axes[1,1].grid(True)

plt.tight_layout()
plt.show()

## Experiment 2: Varying Service Rates

How does service rate affect system performance?

In [ ]:
# Test different service rates
service_rates = [0.3, 0.5, 0.7, 0.9, 1.1, 1.3]
arrival_rate = 1.0  # fixed

results = []
for rate in service_rates:
    print(f"Running simulation with service rate {rate}...")
    result = run_experiment(arrival_rate, rate)
    results.append(result)

df_service = pd.DataFrame(results)
df_service

In [ ]:
# Plot service rate results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0,0].plot(df_service['service_rate'], df_service['avg_wait_time'], 'bo-')
axes[0,0].set_xlabel('Service Rate (cars/min)')
axes[0,0].set_ylabel('Average Wait Time (s)')
axes[0,0].set_title('Wait Time vs Service Rate')
axes[0,0].grid(True)

axes[0,1].plot(df_service['service_rate'], df_service['avg_queue_length'], 'ro-')
axes[0,1].set_xlabel('Service Rate (cars/min)')
axes[0,1].set_ylabel('Average Queue Length')
axes[0,1].set_title('Queue Length vs Service Rate')
axes[0,1].grid(True)

axes[1,0].plot(df_service['service_rate'], df_service['utilization'], 'go-')
axes[1,0].set_xlabel('Service Rate (cars/min)')
axes[1,0].set_ylabel('Utilization')
axes[1,0].set_title('Utilization vs Service Rate')
axes[1,0].grid(True)

axes[1,1].plot(df_service['service_rate'], df_service['throughput'], 'mo-')
axes[1,1].set_xlabel('Service Rate (cars/min)')
axes[1,1].set_ylabel('Throughput (cars/hour)')
axes[1,1].set_title('Throughput vs Service Rate')
axes[1,1].grid(True)

plt.tight_layout()
plt.show()

## Experiment 3: Multiple Queues

How does adding more queues affect performance?

In [ ]:
# Test different numbers of queues
queue_configs = [
    (1, [1]),      # 1 queue, 1 booth
    (2, [1, 1]),   # 2 queues, 1 booth each
    (3, [1, 1, 1]), # 3 queues, 1 booth each
    (2, [2, 2]),   # 2 queues, 2 booths each
    (3, [2, 2, 2]) # 3 queues, 2 booths each
]

arrival_rate = 2.0
service_rate = 0.8

results = []
for num_queues, nodes in queue_configs:
    print(f"Running simulation with {num_queues} queues, nodes: {nodes}...")
    result = run_experiment(arrival_rate, service_rate, num_queues, nodes)
    result['total_booths'] = sum(nodes)
    result['queue_config'] = f"{num_queues}q_{sum(nodes)}b"
    results.append(result)

df_queues = pd.DataFrame(results)
df_queues

In [ ]:
# Plot queue configuration results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

x_labels = df_queues['queue_config']
x_pos = np.arange(len(x_labels))

axes[0,0].bar(x_pos, df_queues['avg_wait_time'])
axes[0,0].set_xticks(x_pos)
axes[0,0].set_xticklabels(x_labels, rotation=45)
axes[0,0].set_ylabel('Average Wait Time (s)')
axes[0,0].set_title('Wait Time by Queue Configuration')

axes[0,1].bar(x_pos, df_queues['avg_queue_length'])
axes[0,1].set_xticks(x_pos)
axes[0,1].set_xticklabels(x_labels, rotation=45)
axes[0,1].set_ylabel('Average Queue Length')
axes[0,1].set_title('Queue Length by Configuration')

axes[1,0].bar(x_pos, df_queues['utilization'])
axes[1,0].set_xticks(x_pos)
axes[1,0].set_xticklabels(x_labels, rotation=45)
axes[1,0].set_ylabel('Utilization')
axes[1,0].set_title('Utilization by Configuration')

axes[1,1].bar(x_pos, df_queues['throughput'])
axes[1,1].set_xticks(x_pos)
axes[1,1].set_xticklabels(x_labels, rotation=45)
axes[1,1].set_ylabel('Throughput (cars/hour)')
axes[1,1].set_title('Throughput by Configuration')

plt.tight_layout()
plt.show()

## Key Insights

From these experiments, we can observe:

1. **Arrival Rate Impact**: Higher arrival rates lead to longer queues and wait times, eventually causing system overload.
2. **Service Rate Impact**: Faster service reduces wait times and queue lengths, improving overall throughput.
3. **Queue Configuration**: More queues and booths generally improve performance, but the benefits depend on the arrival pattern.

Try modifying the parameters above to explore different scenarios!